# Named Entity Recognition for Vietnamese

This notebook does some experiments on NER task for:

- English using spaCy (success ✅)
- Vietnamese using `underthesea` (success, but problematic results 🤔)
- Vietnamese using PhoBERT (failed to load pre-trained weights 💥)

**My Conclusion**: We have nowhere to go but train our own Vietnamese NER model, at least from some pre-trained weights. See more: [Named Entity Recognition](https://github.com/undertheseanlp/NLP-Vietnamese-progress/blob/master/tasks/named_entity_recognition.md).

## English (spaCy)

In [1]:
# Install scaCy
# See more at https://spacy.io/usage

!conda install -c conda-forge spacy -y -q
!python -m spacy download en_core_web_sm

Retrieving notices: ...working... done
Channels:
 - conda-forge
 - rapidsai
 - nvidia
 - nodefaults
 - defaults
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - spacy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    annotated-types-0.7.0      |     pyhd8ed1ab_0          18 KB  conda-forge
    catalogue-2.0.10           |  py310hff52083_0          35 KB  conda-forge
    click-8.1.7                |unix_pyh707e725_0          82 KB  conda-forge
    cloudpathlib-0.20.0        |     pyhd8ed1ab_0          43 KB  conda-forge
    conda-24.11.0              |  py310hff52083_0         907 KB  conda-forge
    confection-0.1.4           |  py310h17c5347_0          66 KB  conda-forge
    cymem-2.0.10               |  py310hf71b8c6_0          50 KB  conda-forge
    cython-blis-1.0.1          |  py310hf46

In [2]:
import spacy
from tqdm import tqdm

In [3]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('sentencizer')

In [4]:
def get_entities_from_doc(doc):
    entities = [e.text for e in doc.ents]
    return entities

def get_entity_types_from_doc(doc):
    types = [e.label_ for e in doc.ents]
    return types

def get_entities(texts, *, n_process=16, batch_size=64):
    docs = nlp.pipe(texts, n_process=n_process, batch_size=batch_size)
    entities = []
    entity_types = []
    for doc in tqdm(docs, total=len(texts)):
        entities.append(get_entities_from_doc(doc))
        entity_types.append(get_entity_types_from_doc(doc))
    return entities, entity_types

if __name__ == '__main__':
    texts = [
        'Today, I\'m travelling to the United States and I will arrive at 5 o\'clock.'
    ]
    entities, entity_types = get_entities(texts)
    for entity, entity_type in zip(entities, entity_types):
        for a, b in zip(entity, entity_type):
            print('%s [%s]' % (a, b))

100%|██████████| 1/1 [00:00<00:00,  3.98it/s]

Today [DATE]
the United States [GPE]
5 o'clock [TIME]


## Vietnamese (underthesea)

In [5]:
!pip install underthesea -q

In [6]:
from underthesea import ner

In [7]:
text = '''
Công ty Cổ phần Dầu Thực vật Tường An (mã CK: TAC) đã công bố BCTC quý 1/2022. 
Cụ thể, doanh thu thuần đạt 1.697 tỷ đồng tăng 7,2% so với cùng kỳ. Tuy nhiên 
giá vốn hàng bán chiếm tới 94% trong doanh thu khiến lãi gộp chỉ còn hơn 98 
tỷ đồng, giảm 55% so với quý 1/2021. Trong kỳ chi phí bán hàng giảm mạnh từ 
113,5 tỷ đồng xuống còn hơn 21 tỷ đồng, chi phí QLDN cũng thấp hơn cùng kỳ. 
Do lãi gộp thấp nên kết quả TAC vẫn báo lãi sau thuế giảm 32% so với cùng kỳ, 
đạt 53 tỷ đồng – tương đương EPS đạt 1.559 đồng.
'''
text = ' '.join(text.split())
info = ner(text)

text, info[:5]

('Công ty Cổ phần Dầu Thực vật Tường An (mã CK: TAC) đã công bố BCTC quý 1/2022. Cụ thể, doanh thu thuần đạt 1.697 tỷ đồng tăng 7,2% so với cùng kỳ. Tuy nhiên giá vốn hàng bán chiếm tới 94% trong doanh thu khiến lãi gộp chỉ còn hơn 98 tỷ đồng, giảm 55% so với quý 1/2021. Trong kỳ chi phí bán hàng giảm mạnh từ 113,5 tỷ đồng xuống còn hơn 21 tỷ đồng, chi phí QLDN cũng thấp hơn cùng kỳ. Do lãi gộp thấp nên kết quả TAC vẫn báo lãi sau thuế giảm 32% so với cùng kỳ, đạt 53 tỷ đồng – tương đương EPS đạt 1.559 đồng.',
 [('Công ty', 'N', 'B-NP', 'O'),
  ('Cổ phần', 'V', 'B-VP', 'O'),
  ('Dầu Thực vật', 'N', 'B-NP', 'B-LOC'),
  ('Tường An', 'Np', 'B-NP', 'I-LOC'),
  ('(', 'CH', 'O', 'O')])

In [8]:
def get_entities(texts):
    results = []
    for text in tqdm(texts, total=len(texts)):
        result = []
        items = ner(text)
        for item in items:
            entity, _, __, entity_type = item
            if entity_type == 'O':
                continue
            result.append((entity, entity_type))
        results.append(result)
    return results

entities = get_entities([text])
entities

100%|██████████| 1/1 [00:00<00:00,  4.59it/s]


[[('Dầu Thực vật', 'B-LOC'),
  ('Tường An', 'I-LOC'),
  ('TAC', 'B-LOC'),
  ('BCTC', 'B-PER'),
  ('chi phí', 'B-LOC'),
  ('QLDN', 'I-LOC'),
  ('kết quả', 'B-LOC'),
  ('TAC', 'I-LOC'),
  ('tương đương', 'B-LOC'),
  ('EPS', 'I-LOC')]]

## Vietnamese (PhoBERT)

In [9]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

def load_phobert_ner():
    model_id = 'vinai/phobert-large'
    model = AutoModelForTokenClassification.from_pretrained(model_id)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    def predict(text):
        inputs = tokenizer(text, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**inputs)
            labels = torch.argmax(outputs.logits, dim=-1)
        token_labels = tokenizer.convert_ids_to_tokens(labels[0].numpy())
        return token_labels
    return predict

predict_vi_ner = load_phobert_ner()

config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
